# Speed-Up the scraping with multiprocessing

In [1]:
#Wikipedia-Article to srape/plot
article = 'Schokolade'

In [2]:
#Import
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests

from multiprocessing import Pool, Manager, Process
import time


main_url = 'https://de.wikipedia.org'
urls_next = [main_url+'/wiki/'+article]

def dataCleaning(title):
    #Cleaning the titles
    return title.replace('_', ' ').replace('%C3%B6', 'ö').replace('%C3%9C', 'Ü').replace('%C3%A4', 'ä').replace('%C3%BC', 'ü').replace('%C3%9F', 'ß')

def processURL(url):
    #Load, scrape and store the data of an url
    data = []
    html = urlopen(url)
    bs = BeautifulSoup(html, 'lxml')
    urls_visited.append(url)
    try:
        #Iterate over each entioned category
        for elem in bs.find('div', {'id':'mw-normal-catlinks'}).findAll('a')[1:]:
            href = elem['href']
            url = url.split('/')[-1].split(':')[-1]
            title = href.split(':')[-1]
            data.append([dataCleaning(url), dataCleaning(title), href])
    except:
        pass
    if data:
        source, target, new_urls = zip(*data)
        processData(source, target, new_urls)

def processData(source, target, new_urls):
    #Store the data/graphedges
    e = [edge for edge in zip(source, target)]
    edges.extend(e)

    urls_next.extend([main_url+url for url in [url for url in new_urls if main_url+url not in urls_visited and main_url+url not in urls_next]])
    
def urlGenerator():
    #Yield a new url
    while urls_next:
        yield urls_next.pop()

In [3]:
start = time.time()

manager = Manager()
urls_next = manager.list([main_url+'/wiki/'+article])
edges = manager.list()
urls_visited = manager.list()

#Map urls to processing cores
while urls_next:
    pool = Pool()
    pool.map(processURL, urlGenerator())

    
ende = time.time()

#Basic progression data
print('Dauer:\t{}'.format(ende-start))
print('Edges:\t{}'.format(len(edges)))
print('VURLs:\t{}'.format(len(urls_visited)))
print('NURLs:\t{}'.format(len(urls_next)))
print('URLs/s:\t{}'.format(len(urls_visited)/(ende-start)))

Dauer:	13.155486583709717
Edges:	27
VURLs:	22
NURLs:	0
URLs/s:	1.672306064850717


In [4]:
#Store the data
df = pd.DataFrame(list(edges), columns=['Source', 'Target']).drop_duplicates()
df.to_csv('build/Wiki_Categories_{}.csv'.format(article), index=False, encoding='utf-8')
df.shape

(26, 2)